# Getting solution information

In this tutorial we will see how to extract information from an optimization model, such as solution information. Gurobi has organized this information in so called *attributes*. Each component of an optimization model, i.e., the [Model object](https://www.gurobi.com/documentation/9.0/refman/py_model.html), the [variables objects](https://www.gurobi.com/documentation/9.0/refman/py_var.html) and the [constraints objects](https://www.gurobi.com/documentation/9.0/refman/py_constr.html) are provided with a method called `getAttr` which can be used to query their specific attributes, corresponding to solution and model information. A central exercise is thus that of understanding which attributes can be queried from each object. The full list of attributes for each object is provided in the Gurobi documentation [here](https://www.gurobi.com/documentation/9.0/refman/attributes.html#sec:Attributes). This page should be kept at hand when working with optimization models.

## An example model

In this tutorial we will consider the classical diet problem. The miminum cost diet is to be found in order to satisfy the daily requirement $R_n$ of a set $\mathcal{N}$ of nutrients such as proteins and vitamins. A set $\mathcal{F}$ of foods are considered. Each food has a cost of $C_f$ per unit (e.g., per gram) and provides an amount $A_{fn}$ of nutrient $n$. Let $x_f$ be the quantity of food $f$ included in the daily diet. The problem can be formulated as follows
$$\min \sum_{f\in\mathcal{F}}C_{f}x_{f}$$
$$\sum_{f\in\mathcal{F}}A_{fn}x_{f} = R_n \qquad \forall n \in\mathcal{N}$$
$$x_{f}\geq 0 \qquad \forall f\in\mathcal{F}$$


In [18]:
from gurobipy import *
# Data
foods = ['apples', 'bananas', 'carrots', 'dates','eggs']
nutrients = ['proteins','vitamin_c','iron']
cost = {'apples':8, 'bananas':10, 'carrots':3, 'dates':20,'eggs':15}
composition = {('apples','proteins'):0.4, 
               ('apples','vitamin_c'):6, 
               ('apples','iron'):0.4, 
               ('bananas','proteins'):1.2, 
               ('bananas','vitamin_c'):10, 
               ('bananas','iron'):0.6,
               ('carrots','proteins'):0.6, 
               ('carrots','vitamin_c'):3, 
               ('carrots','iron'):0.4,
               ('dates','proteins'):0.6, 
               ('dates','vitamin_c'):1, 
               ('dates','iron'):0.2,
               ('eggs','proteins'):12.2, 
               ('eggs','vitamin_c'):0, 
               ('eggs','iron'):2.6
              }
requirements = {'proteins':70,'vitamin_c':60,'iron':12}
# Model
m = Model('diet_problem')
x = m.addVars(foods,name = 'x')
expr = x.prod(cost)
m.setObjective(expr,GRB.MINIMIZE)
const = m.addConstrs((quicksum([composition[f,n] * x[f] for f in foods]) >= requirements[n] for n in nutrients), 'nutri')
# Solving the model
m.optimize()

Optimize a model with 3 rows, 5 columns and 14 nonzeros
Coefficient statistics:
  Matrix range     [2e-01, 1e+01]
  Objective range  [3e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 7e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.825000e+01   0.000000e+00      0s
       3    1.3131148e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.04 seconds
Optimal objective  1.313114754e+02


Now that we have a model and have solved it, let us extract some information.

## Retrieving information from the Model

Let us start by having a look at some of the most important attributes of the `Model` object.
First of all, in the documentation you can see that the `Model` object has a [method `getAttr`](https://www.gurobi.com/documentation/9.0/refman/py_model_getattr.html). This method can be used to query information about the model as well as information about sets of decision variables and constraints. Let us see some examples (keep the [list of attributes](https://www.gurobi.com/documentation/9.0/refman/attributes.html#sec:Attributes) at hand).

In [3]:
# Information about the size of the model
print("The model has ", m.getAttr('NumVars')," decision variables")
print("The model has ", m.getAttr('NumConstrs')," constraints")
print("The model has ", m.getAttr('NumIntVars')," integer variables")
print("The model has ", m.getAttr('NumBinVars')," binary variables")

The model has  5  decision variables
The model has  3  constraints
The model has  0  integer variables
The model has  0  binary variables


In [9]:
# The objective value for the current solution. 
# If the model was solved to optimality, then this attribute gives the optimal objective value. 
print("Best objective value ",m.getAttr('ObjVal'))

# The best available bound (lower bound for minimization problems, upper bound for maximization problems)
print("Best bound ",m.getAttr('ObjBound'))

Best objective value  131.31147540983608
Best bound  131.31147540983608


In [29]:
# How long did it take to solve the problem?
print("Solution time = ",m.getAttr('Runtime')," seconds")

Solution time =  0.07843708992004395  seconds


In [11]:
# What is the final status of the optimization algorithm?
print('Status = ',m.getAttr('Status'))

Status =  2


A status of 2 corresponds to optimal. The statuses are discussed [here](https://www.gurobi.com/documentation/9.0/refman/optimization_status_codes.html#sec:StatusCodes) 

In [12]:
# How many simplex iterations did it perform?
print("# Simplex iterations = ",m.getAttr('IterCount'))

# Simplex iterations =  3.0


As you can notice, in the list of attributes there are attributes which make sense only if the problem solved is a linear program (e.g., the number of Simplex iterations) and other that make sense only if the problem is a MIP (e.g., number of branch and bound nodes explored). In fact, if we query the a MIP attribute from an LP we get an error.

In [14]:
print("# Nodes explored = ",m.getAttr('MIPGap'))

AttributeError: b"Unable to retrieve attribute 'MIPGap'"

The `getAttr` method of the model object can also be used to retrieve primal and dual solution information. 
For example, we can use the method to query the optimal value of the decision variables (provided by the attribute `'x'`). In this case, we pass two arguments: the attribute name (`'x'`) and the `tupledict` of decision variables (in this case `x`). It returns a `tupledict` with the optimal value of the decision variable at each key.

In [21]:
solution = m.getAttr('x',x)
print(solution)
print(type(solution))

{'apples': 0.0, 'bananas': 0.0, 'carrots': 20.0, 'dates': 0.0, 'eggs': 4.754098360655738}
<class 'gurobipy.tupledict'>


Other examples of attributes on the decision variables are

In [22]:
print(m.getAttr('LB',x))
print(m.getAttr('UB',x))
print(m.getAttr('VType',x))
print(m.getAttr('VarName',x))

{'apples': 0.0, 'bananas': 0.0, 'carrots': 0.0, 'dates': 0.0, 'eggs': 0.0}
{'apples': 1e+100, 'bananas': 1e+100, 'carrots': 1e+100, 'dates': 1e+100, 'eggs': 1e+100}
{'apples': 'C', 'bananas': 'C', 'carrots': 'C', 'dates': 'C', 'eggs': 'C'}
{'apples': 'x[apples]', 'bananas': 'x[bananas]', 'carrots': 'x[carrots]', 'dates': 'x[dates]', 'eggs': 'x[eggs]'}


Similarly, we can obtain information on the constraints. The most useful is perhaps the optimal value of the corresponding dual variable. This is given by attribute `'Pi'`. In this case, we pass a `tupledict` of constraints as the second argument.

In [23]:
dual_solution = m.getAttr('Pi',const)
print(dual_solution)

{'proteins': 1.2295081967213115, 'vitamin_c': 0.7540983606557377, 'iron': 0.0}


## Retrieving information from the variables and constraints

The attributes of the variables and constraints which we have queried from the model in a aggregate manner, can also be queried on individual variables and constraints.

In [28]:
for food in foods:
    print(x[food].getAttr('VarName'), " = " , x[food].getAttr('x'))
    
for n in nutrients:
    print(const[n].getAttr('ConstrName'), " = " , const[n].getAttr('Pi'))

x[apples]  =  0.0
x[bananas]  =  0.0
x[carrots]  =  20.0
x[dates]  =  0.0
x[eggs]  =  4.754098360655738
nutri[proteins]  =  1.2295081967213115
nutri[vitamin_c]  =  0.7540983606557377
nutri[iron]  =  0.0


## Shortcuts

Finally, notice that the value of all attributes can be queried directly from the `Model`, `Var` and `Constr` objects.  For example, the following statements are equivalent.

In [32]:
print(m.getAttr('ObjVal'), " is equal to ", m.ObjVal)
print(m.getAttr('Runtime'), " is equal to ", m.Runtime)
for food in foods:
    print(x[food].getAttr('x'), " is equal to " , x[food].x)
    
for n in nutrients:
    print(const[n].getAttr('Pi'), " is equal to " , const[n].Pi)

131.31147540983608  is equal to  131.31147540983608
0.07843708992004395  is equal to  0.07843708992004395
0.0  is equal to  0.0
0.0  is equal to  0.0
20.0  is equal to  20.0
0.0  is equal to  0.0
4.754098360655738  is equal to  4.754098360655738
1.2295081967213115  is equal to  1.2295081967213115
0.7540983606557377  is equal to  0.7540983606557377
0.0  is equal to  0.0
